In [2]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.models import Sequential

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator # Data Augmentation

import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [3]:
# resize all images
IMAGE_SIZE = [244, 244]

In [5]:
train_path = 'Datasets/Train'
valid_path = 'Datasets/Test'

In [6]:
# add preprocessing layers to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], # 3 for RGB channel and 1 for B&W
            weights='imagenet',
            include_top=False) # last layer is to be removed

58892288/58889256 [==============================] - 0s 0us/step


In [7]:
# don't have to train existing weights as it is state-of-art algorithm, it is already training
for layer in vgg.layers:
  layer.trainable = False

In [8]:
# for getting number of classes
folders = glob('Datasets/Train/*')

In [9]:
# our layers, you can add more if you want
x = Flatten()(vgg.output)

predictions = Dense(len(folders), activation='softmax')(x)  # add four categories with softmax which we already find out above

In [11]:
# create model object
model = Model(inputs=vgg.input, outputs=predictions)

# view structure of the model
model.summary()

# tell model what cost and optimization method to use
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 244, 244, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 244, 244, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 244, 244, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 122, 122, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 122, 122, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 122, 122, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 61, 61, 128)       0     

In [13]:
# Image Augmentation
train_datagen = ImageDataGenerator(rescale=1/255.0,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1/255.0)

In [ ]:
# generate the images in directory
training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Datasets/Test',
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode = 'categorical')

In [ ]:
# fit the model
'''r=model.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 5,
                         validation_data = test_set,
                         nb_val_samples = 2000)'''

# fit_generator is used when either we have a huge dataset to fit into our memory 
# or when data augmentation needs to be applied.

r = model.fit_generator(training set,
                        validation_set=test_set,
                        epochs=5,
                        steps_per_epoch=len(training_set),
                        validation_steps=len(test_set)
                        )

In [ ]:
# loss
plt.plot(r.history['loss'], label='train_loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['acc'], label='train_acc')
plt.plot(r.history['val_acc'], label='val_acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save the model
from keras.models import load_model
model.save('facesfeatures_new_model.h5')